## Ch. 02- Programming Probabilistically

In [ ]:
# Import pymc and related code
import arviz as az
import pymc as pm
import preliz as pz

In [ ]:
# Import other "data science libraries"
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 2.1 Probabilistic programming

#### 2.1.1 Flipping coins the PyMC way2.1.1 Flipping coins the PyMC way

In [ ]:
# Initialize repeatable random number generator
rng = np.random.default_rng(123)

In [ ]:
# Generate "fake real data"
trials = 4
theta_real = 0.35 # unknown in a real experiment
data = pz.Binomial(
    n=1,
    p=theta_real).rvs(trials,
                      random_state=rng.integers(np.iinfo(np.int32).max))

In [ ]:
plt.scatter(range(trials), data)
plt.show()

In [ ]:
with pm.Model() as our_first_model:
    θ = pm.Beta('θ', alpha=1., beta=1.)
    y = pm.Bernoulli('y', p=θ, observed=data)
    idata = pm.sample(1000)

### 2.2 Summarizing the posterior

In [ ]:
az.plot_trace(idata)
plt.show()

In [ ]:
az.plot_trace(idata, kind='rank_bars', combined=True)
plt.show()

In [ ]:
az.plot_posterior(idata)
plt.show()

### 2.3 Posterior-based decisions

#### 2.3.1 Savage-Dickey density ratio

In [ ]:
az.plot_bf(idata, var_name='θ', prior=rng.uniform(0, 1, 10000), ref_val=0.5)
plt.show()

#### 2.3.2 Region of Practical Equivalence

In [ ]:
az.plot_posterior(idata, rope=[0.45, 0.55])
plt.show()

In [ ]:
az.plot_posterior(idata, ref_val=0.5)
plt.show()

#### 2.3.3 Loss functions

In [ ]:
# Plot the loss
# The plotting part of this code is from 
# [the chapter 02 code](https://github.com/aloctavodia/BAP3/blob/main/code/Chp_02.ipynb).
grid = np.linspace(0, 1, 200)
θ_pos = idata.posterior['θ']
lossf_a = [np.mean(abs(i - θ_pos)) for i in grid]
lossf_b = [np.mean((i - θ_pos) ** 2) for i in grid]

_, ax = plt.subplots(figsize=(12, 3))
for lossf, c in zip([lossf_a, lossf_b], ['C0', 'C1']):
    mini = np.argmin(lossf)
    ax.plot(grid, lossf, c)
    ax.plot(grid[mini], lossf[mini], 'o', color=c)
    ax.annotate('{:.2f}'.format(grid[mini]),
                (grid[mini], lossf[mini] + 0.03),
                color=c)

    ax.set_yticks([])
    ax.set_xlabel(r'$\hat \theta$')

plt.show()

In [ ]:
# A (silly) assymetric loss function
lossf = []
for i in grid:
    if i < 0.5:
        f = 1 / np.median(θ_pos / np.abs(i**2 - θ_pos))
    else:
        f = np.mean((i - θ_pos) ** 2 + np.exp(-i)) - 0.25

    lossf.append(f)

In [ ]:
# Plot the (silly) asymmetric loss function
mini = np.argmin(lossf)
_, ax = plt.subplots(figsize=(12, 3))
ax.plot(grid, lossf)
ax.plot(grid[mini], lossf[mini], 'o')
ax.annotate('{:.2f}'.format(grid[mini]),
(grid[mini] + 0.01, lossf[mini] + 0.1))
ax.set_yticks([])
ax.set_xlabel(r'$\hat \theta$')

### 2.4 Gaussians all the way down

Gaussians are very appealing. They are easy to work with,
many operations applied to Gaussians return another Gaussian.
Additionally, many natural phenomena can be approximated using
Gaussians. In general, almost every time we measure the average
of something, using a **big enough** sample size, the average
will be distributed as a Gaussian.

Many phenomena are indeed averages. For example, the height
of adults. (Actually, this distribution is a **mixture** of
**two** Gaussians - one for men and one for women.)

Consequently, it is important to learn to build Gaussians,
but also to learn how to relax the normality assumptions.
(This relaxation is surprisingly easy with tools like PyMC).



#### 2.4.1 Gaussian inferences

**Background**

We can use nuclear magnetic Resonance (NMR) to study molecules or
living things such as humans, sunflowers, and yeast. NMR allows one
to measure different **observable** quantities related to **unobservable**
molecular properties. Chemical shift is one of these observable
properties that apply to the nuclei of certain types of atoms.
This problem is an example similar to:

- The height of a group of people
- The average time to travel back home
- The weights of bags or oranges

All these examples have continuous variables and can be thought of as an
average plus a dispersion.

Additionally, if the number of possible values is large enough, we can
approximate it using a Gaussian. For example, the sexual partners of
bonobos, a very promiscuous monkey.

In our example, we have 48 chemical shift value.

- The median is around 53
- The inter-quartile range is about 52 to 55
- Two values "far away" from the resto of the data appear to be outliers.

In [ ]:
# Load the data
data = np.loadtxt('./data/chemical_shifts.csv')

In [ ]:
# Plot the data using a boxplot
_, ax = plt.subplots(figsize=(12, 3))
ax.boxplot(data, vert=False)
plt.show()

We'll forget about the two outlying points. We will further assume that
a Gaussian is a good description of the data. Since know neither the mean
nor the standard deviation, we set priors for both of them. Therefore, a
reasonable model is:

$$
\begin{gather}
\mu \sim \mathcal{U(l, h)} \\
\sigma \sim \mathcal{HN(\sigma_{\sigma})} \\
Y \sim \mathcal{N(\mu, \sigma)}
\end{gather}
$$

where

- $\mathcal{U(l, h)}$ is the Uniform distribution between
  $\mathcal{l}$ and $\mathcal{h}$
- $\mathcal{HN(\sigma_{\sigma})}$ is the Half-Normal distribution
  with scale $\mathcal{\sigma_{\sigma}}$
- $\mathcal{N(\mu, \sigma)}$ is the Gaussian distribution with mean,
  $\mathcal{\mu}$, and standard deviation, $\mathcal{\sigma}$.

Since we do not know the possible values of $\mu$ and $\sigma$ - a typical
situation - we can set priors reflecting our ignorance. For example, we
can set the boundaries of our uniform distribution to be
$\mathcal{l} = 40$ and $\mathcal{h} = 75$: a range **larger** than the
range of the data.

For the Half-Normal, in the absence of more information, we can choose a
large value compared to the **scale** of the data. The following PyMC
code puts details to our model.

In [ ]:
with pm.Model() as model_g:
    mu = pm.Uniform('\u03bc', lower=40, upper=70)
    sigma = pm.HalfNormal('\u03c3', sigma=5)
    Y = pm.Normal('Y', mu=mu, sigma=sigma, observed=data)
    idata_g = pm.sample()

In [ ]:
az.plot_trace(idata_g)
plt.show()

In [ ]:
az.plot_pair(idata_g, kind='kde', marginals=True)
plt.show()

In [ ]:
az.summary(idata_g, kind='stats').round(2)

### Posterior predictive checks

One nice element of the Bayesian toolkit. Once one has calculated
the posterior, $P(\theta | Y)$, one can use that posterior to
generate predictions, $p(\tilde{Y})$.

Although calculating the posterior involves an integral, using PyMC
to get posterior predictive samples involves:

- Calling the function, `sample_posterior_predictive()`
- Passing `InferenceData` as the first object

Additionally, we must pass the `model` object, and we **may** use
the `extend_Inferencedata` argument to add the posterior predictive
samples to the `InferenceData` object.

In [ ]:
pm.sample_posterior_predictive(
    idata_g,
    model=model_g,
    extend_inferencedata=True,
)

Additionally, we can plot the posterior predictive checks.

In [ ]:
az.plot_ppc(idata_g, num_pp_samples=100)
plt.show()

This plot contains

| Curve | Explanation |
| ----- | ----------- |
| White | Kernel Density Estimate (KDE) of the observed data |
| Blue | KDEs computed from each of the 100 posterior predictive samples |
| Orange | Posterior predictive mean |

The blue lines reflect our uncertainty of the predicted data. The plots are
_hairy_ or _wonky_ to reflect our (relative) **lack of data**.
We can make additional observations about our simulated data. For example,
the mean of the simulated data is displaced to the right with a slightly
larger variance that the variance of the actual data.

This discrepancy results from

- Our choice of likelihood
- The two observations away from the bulk of the data

How do we interpret this plot? Is the model wrong or right? Can we use it
or do we need a different model?

Well, it depends. The interpretation of a model and its evaluation and
criticism are **always** context dependent.

The author, based on his experience, believes that this model is a
reasonable enough representation of the data **and** a useful one
for most analyses.

However, we could find other models the better fit the whole data set,
including the two observations that are **far** from the bulk of the
data. We'll see how we can do that next.

### 2.6 Robust inferences

One objection we may have with `model_g` is that we are assuming
a Normal distribution. However, as we have mentioned, our data has two
points away from the bulk of the data. Our choice of a Normal
distribution for the likelihood indirectly assumes that we **do not expect**
to see a lot of data points far away from the bulk of the data.

Since the tails of the Normal distribution fall off quickly as we move
away from the mean, the Normal distribution **is surprised** at the two
"extreme" points.

The distribution "reacts" in two ways:

- It moves its mean towards those two extreme points
- It increases its standard deviation

What can we do? We have at least two options:

We can check for errors in the data; for example,

- During cleaning or preprocessing of the data
- Resulting from the malfunction of the measuring equipment

This conclusion may not be helpful. For example, if the data was actually
collected by someone else, we may not be able to reliable draw
these conclusions.

Another option is to declare the two points outliers and remove theme from
the data. Two common rules for identifying outliers are:

- Any point that falls **below** 1.5 x the IQR (inter-quartile range)
  from the lower quartile or falls **above** 1.5 times  the IQR from
  the upper quartile is considered an outlier.
- Any data point that falls below or above N times the standard deviation
  of the data is considered an outlier. In this situation, N usually has
  a value of 2 or 3.

However, as with **any automatic "method"**, these rules of thumb
**are not perfect** and may result in discarding **valid data points.**

As a general rule, Baysians prefer to encode assumptions directly into the
model by using different priors and likelihood rather than appealing to
_ad hoc_ heuristics such as outlier removal rules.

#### 2.6.1 Degree of normality

One distribution is very similar to a Normal distribution. It has three
parameters and is called the Student's t-distribution:

- A location parameter, $\mu$.
- A scale parameter, $\sigma$.
- A normality parameter, $\nu$. (AKA, the degrees of freedom.)

| When...          | Then, the distribution is the... |
|------------------|----------------------------------|
| $\nu$ = $\infty$ | Normal distribution |
| $\nu$ = 1        | Cauchy or Lorentz distribution |

The parameter, $\nu$, has the range [0, $\infty$]. The lower the
value of $\nu$, the heavier the tails of the distribution.
Alternatively, the lower the value of $\nu$, the higher the
kurtosis. Additionally, if $\nu \leq 1$, the mean value of
the Student's T distribution is not defined. However, remember
that we can **always** calculate an empirical mean.

Similarly, the variance of the Student's T distribution is only defined
for values of $\nu > 2$. Additionally, the **scale** of the Student's T
distribution is **not** the same as its **standard deviation**. However,
the scale and the standard deviation become closer and closer as $\nu$
approaches infinity.


#### 2.6.2 A robust version of the Normal model

We will rewrite the previous model (`model_g`) by replacing the
Gaussian distribution with the Student's T distribution. Because the
Student's T distribution has one more parameter, $\nu$, than the
Gaussian, we need to specify one more prior.

For this particular problem, we chose to use the exponential distribution,
but other distributions restricted to the positive interval could
also work.

Here is a symbolic representation of our model:

$$
\begin{gather}
\mu \sim \mathcal{U(l, h)} \\
\sigma \sim \mathcal{HN(\sigma_\sigma)} \\
\nu \sim Exp(\lambda) \\
Y \sim \mathcal{T(\nu, \mu, \sigma)}
\end{gather}
$$

Let's write this model in PyMC. The only cautionary word here is that,
by default, the Exponential distribution in PyMC is parameterized with
the inverse of the mean.

Note that we will set $\nu$ as an Exponential distribution with a mean
of 30, the distribution looks very similar to a Gaussian. Further, we
can see that **most of the action** happens for relatively small values
of $\nu$. Consequently, we can say that an Exponential prior with a
mean of 30 is a **weakly informative prior**. We generally expect the
mean to be around 30, but it can easily move to smaller or larger values.

Finally, in many problems, estimating $\nu$ is of **no direct interest**.

In [ ]:
# Here's our model
with pm.Model() as model_t:
    mu = pm.Uniform('\u03bc', lower=40, upper=75)
    sigma = pm.HalfNormal('\u03c3', sigma=10)
    nu = pm.Exponential('\u03bd', 1/30)
    y = pm.StudentT('y', nu=nu, mu=mu, sigma=sigma, observed=data)
    idata_t = pm.sample()

In [ ]:
# Plot the trace for `model_t`
az.plot_trace(idata_t)
plt.show()

In [ ]:
# Print the summary of `model_t`
az.summary(idata_t, kind='stats').round(2)

In [ ]:
# Compare these results to those from `model_g`
az.summary(idata_g, kind='stats').round(2)

In [ ]:
# Plot the posterior predictive check
pm.sample_posterior_predictive(
    idata_t,
    model=model_t,
    extend_inferencedata=True
)
ax = az.plot_ppc(idata_t,
            figsize=(12, 4),
            num_pp_samples=100,
            mean=False,
            colors=['C1', 'C0', 'C1']
)
ax.set_xlim(40, 70)

plt.show()

### 2.7 InferenceData

Contains four groups:

- `posterior`
- `posterior_predictive`
- `sample_stats`
- `observed_data`

In [ ]:
model_g

In [ ]:
idata_g

In [ ]:
# For example, `InferenceData` allows one to access the posterior data
posterior = idata_g.posterior
posterior

Many members of `idata_g` return an instance of `xarray`. An `xarray`
instance is similar to a NumPy multidimensional array **with labels**.
This choice allows a user to "ignore" the order of dimensions in
the array. For example, the following code returns the first draw
from chain 0 and from chain 2.

In [ ]:
posterior.sel(draw=0, chain=[0, 2])

We use the `sel` method to select a range of values, like the first
100 draws from all chains.

In [ ]:
posterior.sel(draw=slice(0, 100))

Additionally, one can return the mean for the data variables,
$\mu$ and $\sigma$.

In [ ]:
posterior.mean()

The following code returns the mean **over the draws**; that is,
four values for each of $\mu$ and $\sigma$, one per chain.

In [ ]:
posterior.mean('draw')

Most of the time, we do not care about chains and draws; we just want
the posterior samples. We can get this information using `az.extract`.

In [ ]:
stacked = az.extract(idata_g)
stacked

The `az.extract()` method combines the `chain` and `draw` into a
`sample` coordinate upon which we can perform further operations.

By default, `extract()` operates on the posterior; however, by
specifying other groups with the `group` argument, one can extract data
from other groups.

One can also use `az.extract()` to get a random sample from the posterior.

In [ ]:
az.extract(idata_g, num_samples=100)

We will use the `InferenceData` object throughout this book. These many
usages will help one to get familiar with it and learn more about it.

### 2.8 Groups comparison

A common analysis in statistics is group comparison. For example,

- How well patients respond to a certain drug
- Reduction of car accidents by the introduction of new
  traffic regulations
- Student performance under different teaching approaches

Sometimes, this analysis is framed as hypothesis testing with a goal
of declaring a result to be **statistically significant**. This reliance
on statistical significance can be problematic. Remember,

> Statistical significance is not equivalent to practical significance

Hypothesis testing is "culturally" connected to the concept
of **p-values**. However, interpreting p-values is actually much
more difficult than is typically thought.

Instead of doing hypothesis testing, we will take a different route and
focus on **estimating the effect size**; that is, we want to **quantify
the difference between two groups**. Practically, we will move away from
yes-no questions like "Does it work?" or "Is there any effect?" Instead,
we will ask more nuanced questions like, "How well does it work?" or
"How large is the effect?"


#### 2.8.1 The tips dataset

To explore the subject matter of this section, we will use the "well known"
tips dataset. For this example, different groups are the days of the week.
Notice that **no control group exists.** We can arbitrarily establish a
control group, for example, Thursday, as the reference or control.

We'll start the analysis by loading the dataset as a pandas `DataFrame`.

In [ ]:
tips = pd.read_csv('../data/tips.csv')
tips

Although this `DataFrame` contains many different columns of
information, we are only going to use two columns: day and tip.

Here is the distribution of this data using a ridge plot from `arviz`.

In [ ]:
az.plot_forest(
    tips.pivot(columns='day', values='tip').to_dict('list'),
    kind='ridgeplot',
    hdi_prob=1,
    colors='C1',
    figsize=(12, 4),
)
plt.show()

We will perform a small amount of preprocessing of the data.

In [ ]:
categories = np.array(['Thur', 'Fri', 'Sat', 'Sun'])
tip = tips['tip'].values
idx = pd.Categorical(tips['day'], categories=categories).codes

The model for this problem is almost the same as `model_g`; the only
difference is that now, the values $\mu$ and $\sigma$ are **vectors**
not scalars. PyMC syntax is extremely helpful for this situation.
Instead of writing `for` loops, we can write a vectorized model in a
straightforward notation.

In [ ]:
with pm.Model() as comparing_groups:
    mu = pm.Normal('\u03bc', mu=0, sigma=10, shape=4) # Four days of the week
    sigma = pm.HalfNormal('\u03c3', sigma=10, shape=4)
    y = pm.Normal('y', mu=mu[idx], sigma=sigma[idx], observed=tip)

Notice how we passed a `shape` argument when defining both
`mu` and `sigma`. For `mu`, this means we are defining **four
independent normal distributions** (that is, four different
$\mathcal{N(0, 10)}$ distributions). Similarly, for $\sigma$,
we are defining four independent half-normal distributions,
$\mathcal{HN(0, 10)}$. Additionally, notice our use of `idx`
to index the values of $\mu$ and $\sigma$ that we pass to
the likelihood.

PyMC provides an **alternative syntax** which specifies the

- Coordinates
- Dimensions

The advantage of this alternative is that it allows better integration
with ArviZ.

In this example, we have four values for the means and four values for
the standard deviations. We specify this constraint programmatically by
the parameter, `shape=4`. As a result, the generated `InferenceData`
object will also be indexed with index values 0, 1, 2, and 3. These
index values map to each of our four days ('Thu', 'Fri', 'Sat', 'Sun').

In general, we would expect both the programmer and the user to associate
these integer values with the common days of the week. By using
coordinates and dimensions, both we and ArviZ con use the common labels

- Thu
- Fri
- Sat
- Sun

We specify two coordinates:

- `days` with the dimensions ['Thu', 'Fri', 'Sat', 'Sun']
- `days_flat` containing the same labels but repeated according to the
  order and length that corresponds to each observation.

The coordinate, `days_flat`, will be useful later for
posterior predictive tests.

In [ ]:
coords = {'days': categories, 'days_flat': categories[idx]}

with pm.Model(coords=coords) as comparing_groups:
    mu = pm.HalfNormal('\u03bc', sigma=5, dims='days')
    sigma = pm.HalfNormal('\u03c3', sigma=1, dims='days')
    y = pm.Gamma('y', mu=mu[idx], sigma=sigma[idx],
                 observed=tip, dims='days_flat')

    idata_cg = pm.sample()
    idata_cg.extend(pm.sample_posterior_predictive(idata_cg))

Once the posterior distribution is computed, we can perform all the
analyses that we believe are pertinent. For instance, we can perform
a posterior predictive test. With the help of ArviZ, wo can perform
this test by calling `az.plot_ppc`. We use the `coords` and `flatten`
parameters to get one subplot per day.

In [ ]:
_, axes = plt.subplots(2, 2)
az.plot_ppc(idata_cg, num_pp_samples=100,
            coords={'days_flat': [categories]},
            flatten=[],
            ax=axes)
plt.show()

From the previous figure, we see that the model captures the general
shape of the distribution; however, some details are elusive. This lack
may be due to the relatively small sample size, factors other than day
influencing the tip size, or a combination of both.

For now, we consider that the model is "good enough" and move to
explore the posterior. We can explain the results in terms of their
average values and then find for which days that average is higher.

But there are alternatives. For instance, we may want to express the
results in terms of differences in posterior means. Additionally, we
may want to use some measure of the effect size that is popular with
our audience, such as the probability of superiority or Cohen's d.
In the next section, we explain these alternatves.

#### 2.8.2 Cohen's d

Cohen's d is a common way to measure effect size. It is defined by

$$
\frac {\mu_{2} - \mu_{1}} {\sqrt \frac {\sigma_{1}^2 + \sigma_{2}^2} {2}}
$$

Because we have a posterior **distribution**, we can compute a
**distribution** of Cohen's d. We can compute a single value
for Cohen's d by computing the mean or the median of the distribution.

A Cohen's d can be interpreted as a Z-score (a standard score).
A Z-score is the signed number of standard deviations by which a value
differs from the mean value of the quantity observed or measured.
Consequently, a Cohen's d value of 0.5 can be interpreted as a difference
of 0.5 standard deviations from one group to another.

A very nice web page to explore what different values of Cohen's d
look like is https://rpsychologist.com/d3/cohend. In addition, this page
describes other ways to express an effect size, such as the probability
of superiority.


#### 2.8.3 Probability of superiority

Another way to report the effect size is to report the probability of
a point taken at random from one group has a larger value than another
point taken at random from the other group.

If we assume the data are normally distributed, we can compute the
probability of superiority from Cohen's d using the following expression:

$$
ps = \Phi ( \frac {\delta} {\sqrt {2}} )
$$

where

- $\Phi$ is the cumulative Normal distribution
- $\delta$ is the Cohen's d

If one agrees with the normality assumption, one can calculate the
probability of superiority from Cohen's d. If not, one can compute
the probability of superiority directly **from the posterior** by
taking random samples from two groups and counting how many times
a random value from one group is greater than a random value from
the other group. This calculation is an example of an advantage of
using **Markov Chain Monte Carlo (MCMC)** methods. That is, once we
get samples from the posterior, we can compute **many quantities**
often in ways that are easier than with other methods.

#### 2.8.4 Posterior analysis of mean differences

To conclude our previous discussion, let's compute the posterior
distribution for differences in means, Cohen's d, and the probability
of superiority and combine them into a single plot.

In [ ]:
cg_posterior = az.extract(idata_cg)

dist= pz.Normal(0, 1)

comparisons = [(categories[i], categories[j]) for i in range(4) for j in range(i + 1, 4)]

_, axes = plt.subplots(3, 2, figsize=(13, 9), sharex=True)
for (i, j), ax in zip(comparisons, axes.ravel()):
    means_diff = cg_posterior['\u03bc'].sel(days=i) - cg_posterior['\u03bc'].sel(days=j)

    d_cohen = (means_diff / np.sqrt((cg_posterior['\u03c3'].sel(days=i)**2 +
                                     cg_posterior['\u03c3'].sel(days=j)**2) / 2)).mean().item()

    ps = dist.cdf(d_cohen / (2 ** 0.5))

    az.plot_posterior(means_diff.values, ref_val=0, ax=ax)
    ax.set_title(f'{i} - {j}')
    ax.plot(0, label=f"Cohen's d = {d_cohen:.2f}\nProb sup = {ps:.2f}", alpha=0)
    ax.legend(loc=1)

plt.show()

One way to read this figure is to compare the reference value of zero
difference with the HDI interval. We observe that the HDI interval
**does not** contain the zero difference reference value in a
single comparison: Thursday and Sunday. For all other pairs of days,
we cannot, statistically, rule out a difference of zero.

However, even comparing Sunday and Thursday, we observe that the average
difference is only $0.48. Is that difference large enough to matter?

The short answer: these kinds of questions can only be informed by
statistics; they cannot be answered by statistics. Formally, we would
need to define a loss function or at least the definition of some
threshold value for the effect size which is informed by other values.

### 2.9 Summary

Although Bayesian statistics is simple, fully probabilistic models
often lead to analytically intractable expressions. Although this
tendency has historically been a barrier to widespread adoption of
Bayesian methods, recent mathematics, statistics, physics, and
computer science allow us to solve - at least in principle -
any inference problem.

We introduced the PyMC library. To add a probability distribution
to a model, we only need a single line of code. Using this library,
distributions can be used as priors (unobserved variables) or
likelihoods (observed variables).

Sampling can be achieved with a single line as well. PyMC allows us
to sample the posterior. If everything has gone right, these samples
are representative of the correct posterior distribution. Further,
these samples will be a representation of the logical consequences
of our hypothesize model and our observed data.

We can explore the posterior generated by PyMC using ArviZ to help
us interpret and visualize posterior distributions. One particular
way of using the posterior is to help us make inference decisions by
comparing the ROPE (region of practical equivalence) against the
HDI interval.

Finally, although we only mention this usage in passing, we briefly
mention loss functions. A loss function is a formal way to quantify
the trade-offs and costs associate with making a decision in the
presence of uncertainty.

Although all our examples so far have used a (simple) one-parameter
model, generalizing to an arbitrary number of parameters is trivial
with PyMC. We exemplify how to do this with the Gaussian and Student's
T models. Specifically, we showed how to use the Student's T
distribution to perform robust inference in the presence of "outliers."
In the next chapters, we'll look at how these models can be used as
part of linear regression models.

Additionally, we used a Gaussian model to compare groups. Although this
analysis is sometimes framed in the context of hypothesis testing, we
frame this task as a problem of inferring the effect size.